In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import cufflinks as cf
%matplotlib inline

In [ ]:
train = pd.read_csv('titanic_train.csv')

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cmap = 'viridis') #To view empty values visually

In [ ]:
sns.set_style('whitegrid')

In [ ]:
sns.countplot(x='Survived',hue = 'Pclass',data=train) #To view survival rate with respect to Pclass

In [ ]:
sns.distplot(train['Age'].dropna()) #To view the age range

In [ ]:
cf.go_offline()
train['Age'].iplot(kind='hist') #To view the same above interactively

In [ ]:
plt.figure(figsize=(12,10))
sns.boxplot(x='Pclass',y='Age',data=train) #To view Age Range of each Pclass

In [ ]:
def instill_age(col): #To return mean age according to each Pclass to fill 'na' values
    age = col[0]
    pc = col[1]
    if pd.isnull(age):
        if pc==1:return 37
        elif pc==2:return 29
        else:return 24
    else:return age
train['Age'] = train[['Age','Pclass']].apply(instill_age,axis=1)

In [ ]:
train.drop('Cabin',axis=1,inplace=True) #Due to large percentage of 'na' values, Cabin has been dropped

In [ ]:
train.dropna(inplace=True)

In [ ]:
sex = pd.get_dummies(train['Sex'],drop_first=True)

In [ ]:
embark = pd.get_dummies(train['Embarked'],drop_first=True)

In [ ]:
train = pd.concat([train,sex,embark],axis = 1)

In [ ]:
train.drop(['Name','Ticket'],inplace = True,axis = 1)

In [ ]:
train.drop(['Sex','Embarked'],inplace = True,axis = 1)

In [ ]:
X,y = train.drop('Survived',axis = 1),train['Survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= 101)

In [ ]:
lg = LogisticRegression()

In [ ]:
lg.fit(X_train,y_train)

In [ ]:
pred = lg.predict(X_test)

In [ ]:
print(classification_report(y_test,pred))
confusion_matrix(y_test,pred)